In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


img_board = cv2.imread('assets/5x5.jpg')
img_board = cv2.rotate(img_board, cv2.ROTATE_90_COUNTERCLOCKWISE)
img_board_gray = cv2.cvtColor(img_board, cv2.COLOR_BGR2GRAY)
img_h,img_w = img_board_gray.shape
background_thresh = img_board_gray[0][0]
ADD_THRESH = 90
blur = cv2.GaussianBlur(img_board_gray,(5,5),0)
total_thresh = background_thresh + ADD_THRESH
_,thresh_img = cv2.threshold(blur,total_thresh,255,cv2.THRESH_BINARY)
contours, hier = cv2.findContours(thresh_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
top_25_contours = sorted(contours, key=lambda x : cv2.contourArea(x) if cv2.contourArea(x) < (img_h * img_w)/25 else 0,reverse=True)[:25]

# sort x and y later
coords_and_index = []
for i,contour in enumerate(top_25_contours):
    x, y, _, _ = cv2.boundingRect(contour)
    coords_and_index.append((i,x,y))
print(coords_and_index)
sorted_y = sorted(coords_and_index,key=lambda x:x[2])
print(sorted_y)
for i in range(5):
    sorted_y[5 * i:5* (i + 1)] = sorted(sorted_y[5 * i:5* (i + 1)], key=lambda x:x[1])
top_25_sorted = [top_25_contours[i[0]] for i in sorted_y]
print(sorted_y)


[(0, 206, 2360), (1, 3231, 2308), (2, 1695, 2286), (3, 952, 1829), (4, 956, 2341), (5, 2479, 2279), (6, 199, 1841), (7, 2472, 1787), (8, 3201, 1791), (9, 1705, 1790), (10, 205, 1313), (11, 946, 1277), (12, 2491, 1263), (13, 3208, 1251), (14, 970, 734), (15, 236, 758), (16, 2502, 701), (17, 3201, 694), (18, 218, 196), (19, 1702, 715), (20, 1733, 1260), (21, 1701, 183), (22, 3211, 171), (23, 966, 192), (24, 2439, 159)]
[(24, 2439, 159), (22, 3211, 171), (21, 1701, 183), (23, 966, 192), (18, 218, 196), (17, 3201, 694), (16, 2502, 701), (19, 1702, 715), (14, 970, 734), (15, 236, 758), (13, 3208, 1251), (20, 1733, 1260), (12, 2491, 1263), (11, 946, 1277), (10, 205, 1313), (7, 2472, 1787), (9, 1705, 1790), (8, 3201, 1791), (3, 952, 1829), (6, 199, 1841), (5, 2479, 2279), (2, 1695, 2286), (1, 3231, 2308), (4, 956, 2341), (0, 206, 2360)]
[(18, 218, 196), (23, 966, 192), (21, 1701, 183), (24, 2439, 159), (22, 3211, 171), (15, 236, 758), (14, 970, 734), (19, 1702, 715), (16, 2502, 701), (17, 320

In [43]:
print_img = cv2.cvtColor(thresh_img, cv2.COLOR_BGR2RGB)
cv2.drawContours(print_img, top_25_contours, -1, (0, 255, 0), 3)

font = cv2.FONT_HERSHEY_SIMPLEX

fontScale = 10
color = (255, 0, 0)
thickness = 5
for i, place in enumerate(sorted_y):  
    cv2.putText(print_img, str(i), (place[1] + 10,place[2] + 10), font, 
                   fontScale, color, thickness, cv2.LINE_AA)

imS = cv2.resize(print_img, (960, 540)) 
cv2.imshow('Contours', imS)
cv2.waitKey(0)

-1